In [8]:
import pandas as pd
import pocRegression
import numpy as np

In [37]:
reg_df = pocRegression.makeRegDataset()

In [38]:
reg_df['draftedBig'] = reg_df['drafted']
reg_df['draftedSmall'] = np.where((reg_df['AverageDraftPositionPPR'] > 173) | (reg_df['drafted'] == 0), 0, 1)
reg_df['AverageDraftPositionPPR'] = np.where(reg_df['drafted'], reg_df['AverageDraftPositionPPR'], 190)
reg_df = reg_df[(reg_df ['Year'] > 2015) & (reg_df ['foundAdp'].isin(['left_only', 'both']))].copy()
reg_df['AverageDraftPositionPPRSq'] = reg_df['AverageDraftPositionPPR'] * reg_df['AverageDraftPositionPPR']
reg_df.head()

,Player,Year,FantPos,Pts_PPR,AverageDraftPositionPPR,foundAdp,drafted,PrvYear,QB,RB,TE,WR,draftedBig,draftedSmall,AverageDraftPositionPPRSq
1591,David Johnson,2016,RB,407.80,3.4,both,1,2015,0,1,0,0,1,1,11.56
1592,Ezekiel Elliott,2016,RB,325.40,8.3,both,1,2015,0,1,0,0,1,1,68.89
1593,Aaron Rodgers,2016,QB,380.02,38.7,both,1,2015,1,0,0,0,1,1,1497.69
1594,LeSean McCoy,2016,RB,298.30,22.0,both,1,2015,0,1,0,0,1,1,484.00
1595,Le'Veon Bell,2016,RB,317.40,12.3,both,1,2015,0,1,0,0,1,1,151.29


In [41]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

In [125]:
reg_train, reg_test = train_test_split(reg_df, test_size = 0.2)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
base_vars = ['AverageDraftPositionPPR',
            'AverageDraftPositionPPRSq',
            'draftedBig',
            'draftedSmall',
            'QB','RB','TE','WR'
            ]
base_x = (reg_train[base_vars].copy(), reg_test[base_vars].copy())

y_pts = (reg_train['Pts_PPR'].copy(), reg_test['Pts_PPR'].copy())

In [126]:
def try_lassos(X, y, polys = 2):
    X_train, X_test  = X
    y_train, y_test = y
    X_train = X_train.reset_index().drop('index', axis = 1)
    y_train = np.array(y_train, dtype = float)
    model = make_pipeline(PolynomialFeatures(polys), MaxAbsScaler(), LassoCV(cv = 5, random_state = 0, max_iter = 500000))
    model.fit(X_train, y_train)
    print(f"R^2 score on training data {model.score(X_train, y_train)}")
    print(f"R^2 score on test data {model.score(X_test, y_test)}")
    print()
    return model

adp_pts_model_0 = try_lassos(base_x, y_pts, polys = 2)
adp_pts_model_1 = try_lassos(base_x, y_pts, polys = 3)

R^2 score on training data 0.6399178934480365
R^2 score on test data 0.5953622658681292

R^2 score on training data 0.6420502820571651
R^2 score on test data 0.5959530529775325



array(['1', 'AverageDraftPositionPPR', 'AverageDraftPositionPPRSq',
       'draftedBig', 'draftedSmall', 'QB', 'RB', 'TE', 'WR',
       'AverageDraftPositionPPR^2',
       'AverageDraftPositionPPR AverageDraftPositionPPRSq',
       'AverageDraftPositionPPR draftedBig',
       'AverageDraftPositionPPR draftedSmall',
       'AverageDraftPositionPPR QB', 'AverageDraftPositionPPR RB',
       'AverageDraftPositionPPR TE', 'AverageDraftPositionPPR WR',
       'AverageDraftPositionPPRSq^2',
       'AverageDraftPositionPPRSq draftedBig',
       'AverageDraftPositionPPRSq draftedSmall',
       'AverageDraftPositionPPRSq QB', 'AverageDraftPositionPPRSq RB',
       'AverageDraftPositionPPRSq TE', 'AverageDraftPositionPPRSq WR',
       'draftedBig^2', 'draftedBig draftedSmall', 'draftedBig QB',
       'draftedBig RB', 'draftedBig TE', 'draftedBig WR',
       'draftedSmall^2', 'draftedSmall QB', 'draftedSmall RB',
       'draftedSmall TE', 'draftedSmall WR', 'QB^2', 'QB RB', 'QB TE',
       'QB WR'

In [127]:
polynomial_features = adp_pts_model_0.named_steps['polynomialfeatures']
feature_names = polynomial_features.get_feature_names_out(input_features = base_vars)
print(feature_names)

coefficients = adp_pts_model_0.named_steps['lassocv'].coef_
print(coefficients)

list(zip(feature_names, coefficients))

['1' 'AverageDraftPositionPPR' 'AverageDraftPositionPPRSq' 'draftedBig'
 'draftedSmall' 'QB' 'RB' 'TE' 'WR' 'AverageDraftPositionPPR^2'
 'AverageDraftPositionPPR AverageDraftPositionPPRSq'
 'AverageDraftPositionPPR draftedBig'
 'AverageDraftPositionPPR draftedSmall' 'AverageDraftPositionPPR QB'
 'AverageDraftPositionPPR RB' 'AverageDraftPositionPPR TE'
 'AverageDraftPositionPPR WR' 'AverageDraftPositionPPRSq^2'
 'AverageDraftPositionPPRSq draftedBig'
 'AverageDraftPositionPPRSq draftedSmall' 'AverageDraftPositionPPRSq QB'
 'AverageDraftPositionPPRSq RB' 'AverageDraftPositionPPRSq TE'
 'AverageDraftPositionPPRSq WR' 'draftedBig^2' 'draftedBig draftedSmall'
 'draftedBig QB' 'draftedBig RB' 'draftedBig TE' 'draftedBig WR'
 'draftedSmall^2' 'draftedSmall QB' 'draftedSmall RB' 'draftedSmall TE'
 'draftedSmall WR' 'QB^2' 'QB RB' 'QB TE' 'QB WR' 'RB^2' 'RB TE' 'RB WR'
 'TE^2' 'TE WR' 'WR^2']
[ 0.00000000e+00 -5.00068988e+02  4.62980411e+02  4.14147387e+01
  4.96439555e+01  3.48146021e+01 -2.4

[('1', 0.0),
 ('AverageDraftPositionPPR', -500.06898809643616),
 ('AverageDraftPositionPPRSq', 462.98041120884403),
 ('draftedBig', 41.41473868228322),
 ('draftedSmall', 49.64395547173226),
 ('QB', 34.81460206826312),
 ('RB', -2.4086652320229627),
 ('TE', 1.4047069897538906),
 ('WR', -0.5785876672931045),
 ('AverageDraftPositionPPR^2', 38.73240132827402),
 ('AverageDraftPositionPPR AverageDraftPositionPPRSq', 0.0),
 ('AverageDraftPositionPPR draftedBig', -0.0),
 ('AverageDraftPositionPPR draftedSmall', -233.58713478326007),
 ('AverageDraftPositionPPR QB', -287.79217641723125),
 ('AverageDraftPositionPPR RB', 4.33060772739206),
 ('AverageDraftPositionPPR TE', -23.534939778053808),
 ('AverageDraftPositionPPR WR', -0.0),
 ('AverageDraftPositionPPRSq^2', -152.7458780947764),
 ('AverageDraftPositionPPRSq draftedBig', 85.40067341101154),
 ('AverageDraftPositionPPRSq draftedSmall', 125.38854396250113),
 ('AverageDraftPositionPPRSq QB', 171.68201245343198),
 ('AverageDraftPositionPPRSq RB', 0.

In [146]:
reg_df['pred'] = adp_pts_model_0.predict(reg_df[base_vars])
reg_df['var'] = reg_df['pred'] - reg_df['Pts_PPR']
reg_df['var2'] = (reg_df['pred'] - reg_df['Pts_PPR']) ** 2
reg_df.head()
# reg_df.sort_values(['Year','FantPos'], ascending = [True, False]).head(50)

,Player,Year,FantPos,Pts_PPR,AverageDraftPositionPPR,foundAdp,drafted,PrvYear,QB,RB,TE,WR,draftedBig,draftedSmall,AverageDraftPositionPPRSq,pred,var,var2
1591,David Johnson,2016,RB,407.80,3.4,both,1,2015,0,1,0,0,1,1,11.56,227.448956,-180.351044,32526.498951
1592,Ezekiel Elliott,2016,RB,325.40,8.3,both,1,2015,0,1,0,0,1,1,68.89,219.936032,-105.463968,11122.648539
1593,Aaron Rodgers,2016,QB,380.02,38.7,both,1,2015,1,0,0,0,1,1,1497.69,317.083890,-62.936110,3960.953920
1594,LeSean McCoy,2016,RB,298.30,22.0,both,1,2015,0,1,0,0,1,1,484.00,200.031309,-98.268691,9656.735583
1595,Le'Veon Bell,2016,RB,317.40,12.3,both,1,2015,0,1,0,0,1,1,151.29,213.956822,-103.443178,10700.491151


In [178]:
reg_df['bins'] = reg_df.groupby(['FantPos', 'draftedSmall'])[['AverageDraftPositionPPR']].transform(lambda x: pd.cut(x, bins = 3))
a = reg_df.groupby(['FantPos','draftedSmall','bins']).agg({'var' : ['mean', 'count'], 'var2' : 'mean'}).reset_index()
a['sd'] = np.sqrt(a['var2']['mean'])

# Takeaways?
# For drafted positions, no crazy differences from prediction (var) by position/ PPR-bin
# 


0     3648.621037
1     1710.167368
2     1508.380179
3     4386.420763
4     7153.154899
5     5643.493459
6     1473.383090
7     2759.134192
8      562.123819
9     6930.983072
10    4464.245595
11    3778.207477
12    1194.077864
13    1093.176025
14     420.174861
15    5519.245907
16    3030.967932
17    3152.349957
18    1818.420983
19     890.588781
20    1092.892577
21    5636.885899
22    3911.445857
23    3613.361908
Name: mean, dtype: float64

In [175]:
reg_df.loc[(reg_df['FantPos'] == 'TE') & (reg_df['AverageDraftPositionPPR'] <= 58.03)].sort_values(['Year','AverageDraftPositionPPR'], ascending = [False, True])

,Player,Year,FantPos,Pts_PPR,AverageDraftPositionPPR,foundAdp,drafted,PrvYear,QB,RB,TE,WR,draftedBig,draftedSmall,AverageDraftPositionPPRSq,pred,var,var2,test,bins
4958,Travis Kelce,2022,TE,316.30,16.5,both,1,2021,0,0,1,0,1,1,272.25,222.391362,-93.908638,8818.832286,"(6.747, 45.25]","(6.747, 58.033]"
4991,Mark Andrews,2022,TE,190.50,25.5,both,1,2021,0,0,1,0,1,1,650.25,209.651605,19.151605,366.783985,"(6.747, 45.25]","(6.747, 58.033]"
5180,Kyle Pitts,2022,TE,75.60,31.2,both,1,2021,0,0,1,0,1,1,973.44,201.945056,126.345056,15963.073284,"(6.747, 45.25]","(6.747, 58.033]"
4973,George Kittle,2022,TE,200.50,44.2,both,1,2021,0,0,1,0,1,1,1953.64,185.418877,-15.081123,227.440278,"(6.747, 45.25]","(6.747, 58.033]"
5150,Darren Waller,2022,TE,84.80,50.4,both,1,2021,0,0,1,0,1,1,2540.16,178.051394,93.251394,8695.822481,"(45.25, 83.6]","(6.747, 58.033]"
4369,Travis Kelce,2021,TE,262.80,6.9,both,1,2020,0,0,1,0,1,1,47.61,236.751798,-26.048202,678.508804,"(6.747, 45.25]","(6.747, 58.033]"
4522,Darren Waller,2021,TE,133.50,21.6,both,1,2020,0,0,1,0,1,1,466.56,215.086244,81.586244,6656.315168,"(6.747, 45.25]","(6.747, 58.033]"
4391,George Kittle,2021,TE,198.00,30.4,both,1,2020,0,0,1,0,1,1,924.16,203.009743,5.009743,25.097529,"(6.747, 45.25]","(6.747, 58.033]"
4413,Kyle Pitts,2021,TE,176.60,40.1,both,1,2020,0,0,1,0,1,1,1608.01,190.473348,13.873348,192.469790,"(6.747, 45.25]","(6.747, 58.033]"
4362,Mark Andrews,2021,TE,301.10,50.6,both,1,2020,0,0,1,0,1,1,2560.36,177.819263,-123.280737,15198.140210,"(45.25, 83.6]","(6.747, 58.033]"


In [ ]:
# x_vars = ['Year',
#             'AverageDraftPositionPPR',
#             'AverageDraftPositionPPRSq',
#             'foundAdp',
#             'PrvPts_PPR',
#             'foundLastYearStats',
#             'qbDiff',
#             'Draft',
#             'Age',
#             'PrvYrTmPts',
#             'PlayersAtPosition',
#             'PrvYrPtsShare',
#             'QB','RB','TE','WR']
# x = (reg_train[x_vars].copy(), reg_test[x_vars].copy())